# SQL query from table names - Continued

In [1]:
!pip install openai
from openai import OpenAI
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'dotenv'

In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
print(f"Key sucessfully loaded: {len(OPENAI_API_KEY) > 0}")

Key sucessfully loaded: True


## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "hotels",
  "fields": [
    {
      "nombre": "ID_Hotel",
      "tipo": "INT"
    },
    {
      "nombre": "name",
      "tipo": "VARCHAR"
    },
    {
      "nombre": "rooms",
      "tipo": "INT"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "guests",
  "fields": [
    {
      "nombre": "ID_Guest",
      "tipo": "INT"
    },
    {
      "nombre": "name",
      "tipo": "VARCHAR"
    },
    {
      "nombre": "date_of_birth",
      "tipo": "DATE"
    },
    {
      "nombre": "city",
      "tipo": "VARCHAR"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tableName": "reservations",
  "fields": [
    {
      "nombre": "ID_Res",
      "tipo": "INT"
    },
    {
      "nombre": "arrival",
      "tipo": "DATE"
    },
    {
      "nombre": "departure",
      "tipo": "DATE"
    },
    {
      "nombre": "room_number",
      "tipo": "INT"
    },
    {
      "nombre": "status",
      "tipo": "VARCHAR"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
context = [ {'role':'system', 'content':"""
 create table (
        ID_Hotel INT primary key,
        name VARCHAR,
        rooms INT);

    /*3 example rows
    select * from hotels limit 3;
    ID_Hotel    name                              rooms
    153         ESTREL Berlin                     1200
    8           Steigenberger Airport Frankfurt   933
    59          SI-Suites Stuttgart               52
    */

 create table guests(
        ID_Guest INT primary key,
        name VARCHAR,
        date_of_birth DATE,
        city VARCHAR,
        foreign key(ID_Hotel) references hotel (ID_Hotel));

    /*3 example rows
    select * from employees limit 3;
    ID_Guest    name            date_of_birth   city
    12          Adam Mayer      22/09/1983      Radolfszell
    15          Jasmin Müller   04/01/1992      Mannheim
    19          Rene Nagel      30/08/1995      Oberdupfingen
    */

  create table reservations(
        ID_Res INT primary key,
        arrival DATE,
        departure DATE;
        room_number INT,
        status VARCHAR,
        foreign key(ID_Guest) references guest (ID_Guest),
        foreign key(ID_Hotel) references hotel (ID_Hotel));

    /*3 example rows
    select * from reservations limit 3;
    ID_Res    arrival      departure     room_number  status
    123784    01/01/2025   05/01/2025                 confirmed
    93676     07/10/2024   19/10/2024    210          checked_in
    74673     19/06/2024   22/06/2024    143          checked_out
    */
"""} ]



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.

    Question: Return the names of guests which are born between 10. October 1999 und 10. October 2000?
    SELECT g.name
    FROM guests g
    WHERE g.date_of_birth BETWEEN '1999-10-10' AND '2000-10-10';

    Question: Return a List of checked_in reservations, for a specific named hotel
    SELECT r.ID_Res
    FROM reservations r
    JOIN hotels h ON r.ID_Hotel = h.ID_Hotel
    WHERE r.status = 'checked_in' AND r.name = 'Hotel am Schillerpark';
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [8]:
#new
context_user = context.copy()
print(return_CCRMSQL("""Guest names with more then 3 reservations""", context_user))

SELECT g.name
FROM guests g
JOIN reservations r ON g.ID_Guest = r.ID_Guest
GROUP BY g.ID_Guest
HAVING COUNT(r.ID_Res) > 3;


In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Guest names with more then 3 reservations", old_context_user))

This is your SQL:
```sql
SELECT g.name, COUNT(r.ID_Res) AS num_reservations
FROM guests g
JOIN reservations r ON g.ID_Guest = r.ID_Guest
GROUP BY g.name
HAVING COUNT(r.ID_Res) > 3;
```

This SQL query selects the guest names and counts the number of reservations they have made by joining the "guests" and "reservations" tables on the guest ID. It then groups the results by guest name and filters only those guests who have more than 3 reservations.


In [10]:
#new
print(return_CCRMSQL("Select Reservations which are in-house next new year", context_user))

```sql
SELECT *
FROM reservations
WHERE arrival <= '2023-01-01' AND departure >= '2023-01-01';
```


In [11]:
#old
print(return_CCRMSQL("Select Reservations which are in-house next new year", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM reservations
WHERE arrival <= '2023-01-01' AND departure >= '2023-01-01' AND status = 'in-house';
```

This SQL query selects reservations that are in-house on New Year's Day of the next year (2023-01-01) by checking if the arrival date is before or on New Year's Day and the departure date is on or after New Year's Day, and the status is 'in-house'.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [12]:
#new
print(return_CCRMSQL("Find the names of all hotels with more than 500 rooms.", context_user))

```sql
SELECT name
FROM hotels
WHERE rooms > 500;
```


In [13]:
#old
print(return_CCRMSQL("Find the names of all hotels with more than 500 rooms.", old_context_user))

This is your SQL:
```sql
SELECT name
FROM hotels
WHERE rooms > 500;
```

Explanation: This SQL query selects the names of hotels from the "hotels" table where the number of rooms is greater than 500.


In [14]:
#new
print(return_CCRMSQL("List the reservations that are currently 'checked_in' and are scheduled to depart before '2024-12-31'.", context_user))

```sql
SELECT *
FROM reservations
WHERE status = 'checked_in' AND departure < '2024-12-31';
```


In [15]:
#old
print(return_CCRMSQL("List the reservations that are currently 'checked_in' and are scheduled to depart before '2024-12-31'.", old_context_user))

This is your SQL:
```sql
SELECT * 
FROM reservations
WHERE status = 'checked_in' AND departure < '2024-12-31';
```

This SQL query selects all reservations from the 'reservations' table where the status is 'checked_in' and the departure date is before December 31, 2024.


In [16]:
#new
print(return_CCRMSQL("Get the names of all guests who live in Munich have a reservation at the hotel named 'ESTREL Berlin'.", old_context_user))

This is your SQL:
```sql
SELECT g.name
FROM guests g
JOIN reservations r ON g.ID_Guest = r.ID_Guest
JOIN hotels h ON r.room_number = h.ID_Hotel
WHERE g.city = 'Munich' AND h.name = 'ESTREL Berlin';
```

This SQL query retrieves the names of guests who live in Munich and have a reservation at the hotel named 'ESTREL Berlin'. It achieves this by joining the `guests`, `reservations`, and `hotels` tables on their respective keys and filtering the results based on the specified conditions.


In [17]:
#old
print(return_CCRMSQL("Get the names of all guests who live in Munich have a reservation at the hotel named 'ESTREL Berlin'.", old_context_user))

This is your SQL:
```sql
SELECT g.name
FROM guests g
JOIN reservations r ON g.ID_Guest = r.ID_Guest
JOIN hotels h ON r.room_number = h.ID_Hotel
WHERE g.city = 'Munich' AND h.name = 'ESTREL Berlin';
```

This SQL query retrieves the names of guests who live in Munich and have a reservation at the hotel named 'ESTREL Berlin'. It achieves this by joining the `guests`, `reservations`, and `hotels` tables on their respective keys and filtering the results based on the specified conditions.


**Findings:**

The API does it's best, but is sometimes confused.

When I ask for in-house guests during next new year, with both contexts I received the select with the wrong year.
The old_context even added a statement which would create a wrong result.

For the last tast, both contexts failed, because they selected Reservations of a Hotel by the room_number and not by the ID_Hotel